In [29]:
import pathlib
import pandas as pd
import json

# Encode e-clip data

[eCLIP protocol](https://www.encodeproject.org/documents/dde0b669-0909-4f8b-946d-3cb9f35a6c52/@@download/attachment/eCLIP_analysisSOP_v1.P.pdf)

In [5]:
encode_eclip_path = pathlib.Path('/Volumes/biodb/encode')

In [7]:
encode_metadata = pd.read_table(encode_eclip_path / 'encode_hg38_clip_peaks/metadata.tsv')

In [16]:
encode_metada.head()

,File accession,File format,Output type,Experiment accession,Assay,Biosample term id,Biosample term name,Biosample type,Biosample life stage,Biosample sex,...,dbxrefs,File download URL,Assembly,Platform,Controlled by,File Status,Audit WARNING,Audit INTERNAL_ACTION,Audit NOT_COMPLIANT,Audit ERROR
0,ENCFF743GIA,bed narrowPeak,peaks,ENCSR867DSZ,eCLIP,EFO:0002067,K562,immortalized cell line,adult,female,...,NaN,https://www.encodeproject.org/files/ENCFF743GI...,hg19,NaN,NaN,released,NaN,NaN,NaN,NaN
1,ENCFF392QAQ,bed narrowPeak,peaks,ENCSR867DSZ,eCLIP,EFO:0002067,K562,immortalized cell line,adult,female,...,NaN,https://www.encodeproject.org/files/ENCFF392QA...,hg19,NaN,NaN,released,NaN,NaN,NaN,NaN
2,ENCFF177GMP,bed narrowPeak,peaks,ENCSR867DSZ,eCLIP,EFO:0002067,K562,immortalized cell line,adult,female,...,NaN,https://www.encodeproject.org/files/ENCFF177GM...,GRCh38,NaN,NaN,released,NaN,NaN,NaN,NaN
3,ENCFF669BCV,bed narrowPeak,peaks,ENCSR867DSZ,eCLIP,EFO:0002067,K562,immortalized cell line,adult,female,...,NaN,https://www.encodeproject.org/files/ENCFF669BC...,GRCh38,NaN,NaN,released,NaN,NaN,NaN,NaN
4,ENCFF774EDY,bed narrowPeak,peaks,ENCSR513NDD,eCLIP,EFO:0001187,HepG2,immortalized cell line,child,male,...,NaN,https://www.encodeproject.org/files/ENCFF774ED...,hg19,NaN,NaN,released,NaN,NaN,NaN,NaN


In [22]:
len(list(pathlib.Path('/Volumes/biodb/encode/encode_hg38_clip_peaks/HepG2/combined').glob('*.bed')))

70

In [31]:
with open('/Volumes/prj/dorina2/regulators/h_sapiens/hg38/ICLIP_HNRNPC_hg19.json') as fin:
    print(json.load(fin))

[{'id': 'ICLIP_HNRNPC_hg19', 'experiment': 'ICLIP', 'summary': 'hnRNPC iCLIP (Koenig 2010)', 'description': ' Liftover to hg38 assembly with crossmap.', 'methods': '', 'references': []}]


In [36]:
with open('/Volumes/prj/dorina2/regulators/h_sapiens/hg38/CLASH_mirna_hg19.json') as fin:
    x = json.load(fin)

In [61]:
eclip_metadata = []

template = {
    'description': 'encode eClip data.',
     'experiment': 'eClip',
     'id': 'eClip_all',
     'methods': '',
     'references': 'Nostrand E, et al. bioRxiv 179648; doi: https://doi.org/10.1101/179648.',
     'summary': 'all combined eClip sites from K562 and HepG2.'
}
eclip_metadata.append(template)

In [62]:
encode_eclip_path = pathlib.Path('/Volumes/biodb/encode/encode_hg38_clip_peaks/HepG2/combined')
data = []
col_names = ["chrom","start","end","name","score","strand","signal_value","-log10(pValue)","-log10(qvalue)","peak"]

for f in encode_eclip_path.glob('*.bed'):
    tmp_template = template.copy()
    tmp_template['id'] =  f.stem
    
    x = pd.read_table(f, names=col_names)
    x['.'] = '.'
    x['name'] = x['name'] + '|eClip'
    data.append(x)
    
    tmp_template['summary'] = "{} peaks for targets of {} in {} cells.".format(
        x.shape[0], f.stem.split('_')[0], f.stem.split('_')[1])
    eclip_metadata.append(tmp_template)    

In [64]:
data = pd.concat(data)

In [68]:
data[["chrom", "start", "end", "name", ".", "strand"]].to_csv(
    '/Volumes/prj/dorina2/regulators/h_sapiens/hg38/eClip_RBP_hg38.bed', 
    index=None, header=None, sep="\t")

In [71]:
with open('/Volumes/prj/dorina2/regulators/h_sapiens/hg38/eClip_RBP_hg38.json', 'w') as fout:
    json.dump(eclip_metadata, fout, indent=True)